# Test of LSDB critical functions

This notebook invokes several critical methods, necessary for the success of most users of LSDB within RSP. This does not rely on any pre-existing outside datasets, and only requires pip installation of the `lsdb` package and its dependencies.

Flow is as follows:
- generate two small, in-memory catalogs
- write the two catalogs to disk
- load the two catalogs from disk
- crossmatch the catalogs
- perform a basic summation operation

In [1]:
import lsdb
import tempfile
from pathlib import Path

tmp_path = tempfile.TemporaryDirectory()
tmp_dir = Path(str(tmp_path))

In [2]:
region = lsdb.ConeSearch(ra=53.13, dec=-28.10, radius_arcsec=20)
expected_pixels = [lsdb.HealpixPixel(7, 143742)]

gen_catalog_1 = lsdb.generate_catalog(100, 5, seed=53, search_region=region)
assert gen_catalog_1.get_healpix_pixels() == expected_pixels
gen_catalog_1.to_hats(tmp_dir / "catalog_1", catalog_name="catalog_1")

gen_catalog_2 = lsdb.generate_catalog(100, 5, seed=28, search_region=region)
assert gen_catalog_2.get_healpix_pixels() == expected_pixels
gen_catalog_2.to_hats(tmp_dir / "catalog_2", catalog_name="catalog_2")

In [3]:
catalog_1 = lsdb.open_catalog(tmp_dir / "catalog_1")
catalog_2 = lsdb.open_catalog(tmp_dir / "catalog_2")

In [4]:
crossmatch_result = lsdb.crossmatch(catalog_1, catalog_2)
assert len(crossmatch_result.compute()) > 0

/home/delucchi/git/rsp/lsdb/src/lsdb/dask/crossmatch_catalog_data.py:147: RuntimeWarning: Right catalog does not have a margin cache. Results may be incomplete and/or inaccurate.
  warnings.warn(


In [5]:
def sum_id(df):
    import pandas as pd

    return pd.DataFrame([{"sum": df["id"].sum()}])


unrealized = catalog_1.map_partitions(sum_id)
assert unrealized.compute()["sum"][0] == 52752

unrealized = catalog_2.map_partitions(sum_id)
assert unrealized.compute()["sum"][0] == 56349

In [6]:
tmp_path.cleanup()